In [1]:
import pytesseract
import cv2
import boto3
import tqdm
from dotenv import load_dotenv
import os

In [2]:
load_dotenv()

True

In [3]:
image_folder = 'samples/'
output_folder = 'processed/'

In [4]:
os.listdir(image_folder)

['20240624_083054.jpg',
 '20240709_144534.jpg',
 '20240627_113950.jpg',
 '20240411_134000.jpg',
 'IMG-20240417-WA0033.jpg',
 '20240629_111138.jpg',
 '20240709_115613.jpg']

In [5]:
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for file in os.listdir(image_folder):
    file_path = os.path.join(image_folder,file)
    out_path = os.path.join(output_folder,f'processed_{file}')
    img = cv2.imread(file_path)
    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    height,width = img.shape
    roi = img[int(height * 0.5) : height,0:int(width*0.6)]
    cv2.imwrite(out_path,roi)


In [6]:
attributes= {}
total_text = []

In [11]:
session = boto3.Session(
    aws_access_key_id=os.environ.get('AWS_ACCESS_ID'),
    aws_secret_access_key=os.environ.get('AWS_PASS_KEY'),
    region_name=os.environ.get('REGION')
)
client = session.client('textract',region_name='us-east-1')

In [12]:
for file in tqdm.tqdm(os.listdir(output_folder)):
    file_path = os.path.join(output_folder,file)
    with open(file_path,'rb') as file:
        image = file.read()
        byte_image = bytearray(image)
        print(f'file loaded ---> {file_path}')
    response = client.detect_document_text(Document={"Bytes": byte_image})
    temp =[]
    for i in response['Blocks']:
        if i['BlockType'] == 'LINE':
            # print(i['BlockType'])
            temp.append(i['Text'])
    total_text.append(temp)

  0%|          | 0/7 [00:00<?, ?it/s]

file loaded ---> processed/processed_20240624_083054.jpg


 14%|█▍        | 1/7 [00:04<00:29,  4.96s/it]

file loaded ---> processed/processed_20240411_134000.jpg


 29%|██▊       | 2/7 [00:11<00:29,  5.90s/it]

...

 71%|███████▏  | 5/7 [00:21<00:07,  3.89s/it]

file loaded ---> processed/processed_20240629_111138.jpg


 86%|████████▌ | 6/7 [00:23<00:03,  3.33s/it]

file loaded ---> processed/processed_20240709_144534.jpg


100%|██████████| 7/7 [00:26<00:00,  3.84s/it]


In [14]:
for i in total_text:
    print(i)

['Latitude: 19.579036', 'Longitude: 76.275787', 'Elevation: 449.34+16 m', 'Accuracy: 21.2 m', 'Time: 24-06-2024 08:30', 'Note: Farmer Id:-42223', 'Farmer Name: Annasaheb Vitthalrao Kharat', 'Crop:- Banana Callus Saplings:- 3000 Village:']
['Latitude: 17.821182', 'Longitude: 75.014548', 'Elevation: 537.05+4 m', 'Accuracy: 1.4 m', '111', 'Time: 11-04-2024 01:39 PM', 'Note: Farmer ID : 41730', 'Farmer Name : Archana Pratap Rachkar', 'Crop Name : Banana G9 Tissue Sa plings', '25000']
['Latitude: 19.333059', 'Longitude: 77.375199', 'Elevation: 399.3+3 m', 'Accuracy: 3.9 m', 'Azimuth: 85° (E)', 'Pitch: 0.5°', 'Time: 06-27-2024 11:39', 'Note: FARMER ID- 42176', 'FARMER NAME- ANJANABAI BAPURAV DHUMAL', 'BANANA DISTRIBUTION PLANTS SAPLING 1500 SHENNITO-A']
['atitude: 18.226968', 'ngitude: 75.229308', 'evation: 468.9+20 m', 'ccuracy: 3.9 m', 'zimuth: 117° (SE)', 'tch: -8.4°', 'me: 17 April 2024 8:59 pm', "te: Farmer I'd - 41641", 'armer Name - Baban Laxman More', 'a GO Sanlina 2600']
['atitude: 